# Train an Expert Agent in AGiXT with ezLocalai Provider

This example assumes that you have your AGiXT and ezLocalai servers set up and running. This example is specifically for running AGiXT with ezLocalai and training a local agent. If you do not wish to run local models, we have the same example set up for using an agent with the OpenAI API.

If you do not have ezLocalai set up and you want to set it up to run local models, go to https://github.com/DevXT-LLC/ezlocalai and follow the instructions there to set up the server, then continue with this example.

## Create the Agent

For this example, we will create an expert on AGiXT. These settings can be easily changed in the streamlit app or over API.

Modify the `agixt_server`, `api_key`, `agent_name`, `EZLOCALAI_API_URL`, `EZLOCALAI_API_KEY`, `persona`, and any others as needed.

In [ ]:
from agixtsdk import AGiXTSDK

agixt_server = "http://localhost:7437"  # Change this to your AGiXT server URL
api_key = "None"  # Change this to your AGiXT API key

agixt = AGiXTSDK(base_uri=agixt_server, api_key=api_key)

agent_name = "AGiXT"  # Change this if desired

agixt.add_agent(
    agent_name=agent_name,
    settings={
        "provider": "ezlocalai",  # LLM Provider
        "transcription_provider": "ezlocalai",  # Voice transcription provider, default uses the built in transcription in AGiXT.
        "translation_provider": "ezlocalai",  # Voice translation provider, default uses the built in translation in AGiXT.
        "embeddings_provider": "default",  # Embeddings provider, default uses the built in embeddings in AGiXT.
        "image_provider": "None",  # If set, AGiXT will autonomously create images if it chooses to do so based on the conversation.
        "vision_provider": "ezlocalai",  # Vision provider, None means no vision capabilities. We will use OpenAI's since we're using GPT-4o.
        "tts_provider": "ezlocalai",  # Change this to `default` or whichever TTS provider you want to use. None means no voice response.
        "VOICE": "Morgan_Freeman",  # Voice for TTS, change this to the voice you want to use from ezlocalai.
        "AI_MODEL": "ezlocalai",  # It doesn't matter which model you put here, ezlocalai uses the model it was started with.
        "EZLOCALAI_API_URI": "http://ezlocalai:8091/v1/",  # URL for the EZLOCALAI API, change this to your EZLOCALAI API URL. Never use localhost here, it is a different container.
        "EZLOCALAI_API_KEY": "Your EZLOCALAI API key",  # Change this to your EZLOCALAI API key
        "MAX_TOKENS": 4096,
        "AI_TEMPERATURE": "0.7",
        "AI_TOP_P": "0.95",
        "mode": "prompt",  # For info about chat completion modes, go to https://josh-xt.github.io/AGiXT/2-Concepts/04-Chat%20Completions.html
        "prompt_name": "Chat",
        "prompt_category": "Default",
        "persona": "AGiXT is an expert on the AGiXT AI agent automation platform and supports the users of AGiXT.",  # Use this field to set persona for the AI model
        "context_results": 20,  # How many memories from training to inject with each interaction.
    },
)

## Zip your training data

Creates a zip file called `training_data.zip` of the AGiXT `docs` folder. You can change this to any folder that you would like to use as training data, or skip this step and use an existing zip file.

A good example of what to use for training data would be any PDF, word document, text file, or any other kind of file with information in it that you would like the agent to learn from.

In [1]:
from zipfile import ZipFile
import os

os.chdir("../")
with ZipFile("examples/training_data.zip", "w") as zipObj:
    for foldername, subfolders, filenames in os.walk("docs"):
        for filename in filenames:
            file_path = os.path.join(foldername, filename)
            zipObj.write(file_path)
os.chdir("examples/")

## Train the Agent on the training data

This will train the agent on the training data that you have provided. This will take some time to complete depending on the size of the training data. A zip file around 70MB in size takes around 3 minutes to complete. The AGiXT docs should complete very quickly since it is all markdown files totaling around 3MB.

In [ ]:
import base64

zip_file_name = "training_data.zip"
training_data = base64.b64encode(open(zip_file_name, "rb").read()).decode("utf-8")

agixt.learn_file(
    agent_name=agent_name,
    file_name=zip_file_name,
    file_content=training_data,
    collection_number=0,
)

## Chat with your trained expert AGiXT agent

AGiXT has direct support for using the OpenAI API for chat completions. See this link for more information to take advantage of the abilities of this endpoint: https://josh-xt.github.io/AGiXT/2-Concepts/04-Chat%20Completions.html


In [ ]:
import openai

prompt = "What can you tell me about AGiXT?"

openai.base_url = f"{agixt_server}/v1/"
openai.api_key = api_key

response = openai.chat.completions.create(
    model=agent_name,
    messages=[{"role": "user", "content": prompt}],
    user="Tell me about AGiXT",  # This field is used for the conversation name, if empty, it will use today's date
)
print(response.choices[0].message.content)

# That is all! 

You now have a trained expert agent in AGiXT. This agent will be able to support users by answering questions, providing information, and more about AGiXT.